In [1]:
import datetime as dt
from nytimes_scraper.nyt_api import NytApi
from nytimes_scraper.articles import fetch_articles_by_month, articles_to_df
from nytimes_scraper.comments import fetch_comments, fetch_comments_by_article, comments_to_df
from nytimes_scraper import run_scraper, scrape_month

api = NytApi('FYkXHSbZsX1UXEq8eA7LLhrO1h6m2GSv')


In [2]:
# # # Fetch articles of a specific month
# articles = fetch_articles_by_month(api, dt.date(2020, 2, 1))
# article_df = articles_to_df(articles)

# # Fetch comments from multiple articles
# # a) using the results of a previous article query
# article_ids_and_urls = list(article_df['web_url'].iteritems())
# comments_a = fetch_comments(api, article_ids_and_urls)
# comment_df = comments_to_df(comments_a)

# # b) using a custom list of articles
# comments_b = fetch_comments(api, article_ids_and_urls=[
#     ('nyt://article/316ef65c-7021-5755-885c-a9e1ef2cfdf2', 'https://www.nytimes.com/2020/01/03/world/middleeast/trump-iran-suleimani.html'),
#     ('nyt://article/b2d1b802-412e-51f7-8864-efc931e87bb3', 'https://www.nytimes.com/2020/01/04/opinion/impeachment-witnesses.html'),
# ])

# # Fetch comment for one specific article by its URL
comments_c = fetch_comments_by_article(api, 'https://www.nytimes.com/2023/05/30/technology/ai-threat-warning.html')

In [23]:
commentID = []
userDisplayName = []
userLocation = []
commentBody = []
createDate = []
recommendations = []
replyCount = []
replies = []
parent = []

import pandas as pd

for comment in comments_c:
    commentID.append(comment['commentID'])
    userDisplayName.append(comment['userDisplayName'])
    userLocation.append(comment['userLocation'])
    commentBody.append(comment['commentBody'])
    createDate.append(comment['createDate'])
    recommendations.append(comment['recommendations'])
    replyCount.append(comment['replyCount'])
    replies.append(comment['replies'])
    parent.append(comment['parentID'])
    if comment['replyCount'] > 0: 
        for reply in comment['replies']:
            commentID.append(reply['commentID'])
            userDisplayName.append(reply['userDisplayName'])
            userLocation.append(reply['userLocation'])
            commentBody.append(reply['commentBody'])
            createDate.append(reply['createDate'])
            recommendations.append(reply['recommendations'])
            replyCount.append(reply['replyCount'])
            replies.append(reply['replies'])
            parent.append(reply['parentID'])



df = pd.DataFrame([commentID,userDisplayName,userLocation,commentBody,createDate,recommendations,replyCount,replies,parent]).T
df.columns = ['comment_id','name','location','comment','time','likes','replies_count','replies','parent_id']
df

,comment_id,name,location,comment,time,likes,replies_count,replies,parent_id
0,125405146,Paul,Milwaukee,It is very possible that social media have alr...,1685446635,522,5,"[{'commentID': 125405281, 'status': 'approved'...",None
1,125405281,Chris V,Detroit,@Paul I believe that accurate beliefs are abso...,1685447192,61,0,[],125405146
2,125405707,Paul,Milwaukee,@Chris V Our cognitive systems are not designe...,1685448702,22,0,[],125405281
3,125413598,Chris Burks,"Mount Vernon, WA",@Paul \nAbsolutely agree. I don’t think even l...,1685468267,12,0,[],125405146
4,125413969,L Kim,Seattle,@Paul Belief is a a trait that developed late ...,1685469207,1,0,[],125405146
...,...,...,...,...,...,...,...,...,...
1443,125431033,C,"N.,Y,",Purdue Pharma's Sackler family paid $6 billion...,1685537879,218,0,[],None
1444,125430839,RjW,RollingPrairie,"Unless prevented, the nihilistic qualities of ...",1685537337,39,0,[],None
1445,125428441,LB,U.S.,"Putting ""the"" government in charge of things i...",1685528844,100,0,[],None
1446,125426057,JL Turriff,Concordia MO,This is just another example of 'extinction by...,1685503010,157,0,[],None


In [24]:
df.time = df.time.apply(lambda x: dt.datetime.fromtimestamp(int(x)))
df.time = df.time.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))

In [27]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

def preprocess(text):
    t = text.lower()
    # t = re.sub('_',r'',t)
    # t = re.sub('\d+',r'',t)
    t = re.sub(r'@[^ ]*',r'',t)
    t = re.sub(r'\W+',r' ',t)
    t = re.sub(r'(could|would|like|)', '', t)
    t = re.sub(r'\b(?!(?:ai)\b)\w{3}\b','', t)
    t = re.sub(r'humans',r'human',t)
    stopwords_list = stopwords.words('english')
    txt = ' '.join([word for word in t.split() if word not in stopwords_list])
    return txt

lemmatizer = WordNetLemmatizer()



df['pre_com'] = [' '.join([lemmatizer.lemmatize(preprocess(email))])
                 .strip() for email in df['comment']]

In [28]:
df.to_csv('expanded_comments.csv')